In [ ]:
!git clone https://github.com/gianfrancodemarco/mm-cot
!pip install -r mm-cot/requirements.txt

In [ ]:
import requests
import zipfile
import os

from tqdm import tqdm
import requests

def download_file(url, local_filename="file.foo"):
  # Streaming, so we can iterate over the response.
  response = requests.get(url, stream=True)
  total_size_in_bytes= int(response.headers.get('content-length', 0))
  block_size = 1024 #1 Kibibyte
  progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
  with open(local_filename, 'wb') as file:
      for data in response.iter_content(block_size):
          progress_bar.update(len(data))
          file.write(data)
  progress_bar.close()
  if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
      print("ERROR, something went wrong")

def unzip(file, dest): 
  with zipfile.ZipFile(file, 'r') as zip_ref:
      zip_ref.extractall(dest)

# Download data
!git clone https://github.com/lupantech/ScienceQA
!mv ScienceQA/data data
!rm -r ScienceQA

# Download vision features
download_file(
    'https://drive.google.com/u/0/uc?id=13B0hc_F_45-UlqPLKSgRz-ALtFQ8kIJr&export=download&confirm=t&uuid=a234e603-9c79-4123-9d35-09a326c923d9&at=ALgDtsyNOCvILf1Ri9_B4KcZzjez:1676315895503',
    'vision_features.zip'
)

os.mkdir('vision_features')
unzip('vision_features.zip', '.')

# Download models
download_file(
    'https://drive.google.com/u/0/uc?id=1FtTYOJPHnWnFfCxNC6M3gar4RAX5E21b&export=download&confirm=t&uuid=a5d17c71-252a-421b-9fd0-6ce609da7947&at=ALgDtsxrRDAAnvFetgzK3lXuZ9I_:1676314225641',
    'models.zip'
)

os.mkdir('models')
unzip('models.zip', '.')


Cloning into 'ScienceQA'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 40 (delta 4), reused 35 (delta 3), pack-reused 0
Unpacking objects: 100% (40/40), 10.75 MiB | 8.14 MiB/s, done.


100%|██████████| 1.32G/1.32G [00:26<00:00, 50.6MiB/s]
100%|██████████| 1.66G/1.66G [00:30<00:00, 54.1MiB/s]


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# rationale generation
!CUDA_VISIBLE_DEVICES=0,1 python mm-cot/main.py \
    --model allenai/unifiedqa-t5-base \
    --user_msg rationale --img_type detr \
    --bs 8 --eval_bs 2 --eval_acc 10 --output_len 512 \
    --final_eval --prompt_format QCM-LE \
    --evaluate_dir models/MM-CoT-UnifiedQA-base-Rationale

2023-02-20 15:54:34.216895: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 15:54:35.072703: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-20 15:54:35.072821: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-20 15:54:35.072852: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [ ]:
# Replace console.log in main.py with print

# answer inference
!CUDA_VISIBLE_DEVICES=0,1 python mm-cot/main.py \
    --model allenai/unifiedqa-t5-base \
    --user_msg answer --img_type detr \
    --bs 8 --eval_bs 4 --eval_acc 10 --output_len 64 \
    --final_eval --prompt_format QCMG-A \
    --eval_le models/MM-CoT-UnifiedQA-base-Rationale/predictions_ans_eval.json \
    --test_le models/MM-CoT-UnifiedQA-base-Rationale/predictions_ans_test.json \
    --evaluate_dir models/MM-CoT-UnifiedQA-base-Answer